https://adventofcode.com/2023/day/14

In [1]:
with open("data/14.txt") as fh:
    puzzle = fh.read()

In [2]:
testdata = """\
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""

In [23]:
def parse_data(data):
    rounds = set()
    flats = set()
    for r, line in enumerate(data.splitlines()):
        for c, char in enumerate(line):
            pos = c - r * 1j
            if char == "O":
                rounds.add(pos)
            elif char == "#":
                flats.add(pos)
    return rounds, flats, r+1, c+1

In [148]:
def platform_to_string(rounds, flats, height, width):
    lines = []
    for r in range(height):
        line = []
        for c in range(width):
            pos = c - r * 1j
            line.append("O" if pos in rounds else "#" if pos in flats else ".")
        lines.append(" ".join(line))
    return "\n".join(lines)

In [134]:
def tilt_north(rounds, flats, height, width):
    for r in range(1, height):
        for c in range(width):
            pos = c - r * 1j
            above = pos + 1j
            if pos in rounds and above not in rounds and above not in flats:
                while True:
                    nextabove = above + 1j
                    if nextabove in rounds or nextabove in flats or nextabove.imag > 0:
                        break
                    above = nextabove
                rounds.remove(pos)
                rounds.add(above)

In [135]:
rounds, flats, height, width = parse_data(testdata)
tilt_north(rounds, flats, height, width)

In [149]:
print(platform_to_string(rounds, flats, height, width))

O . . . . # . . . .
O . O O # . . . . #
. . . . . # # . . .
O O . # O . . . . O
. O . . . . . O # .
O . # . . O . # . #
. . O . . # O . . O
. . . . . . . O . .
# . . . . # # # . .
# O O . . # . . . .


In [137]:
def sum_rockloads(rounds, height):
    return sum(height + rock.imag for rock in rounds)

In [138]:
sum_rockloads(rounds, height)

136.0

In [139]:
def total_load_on_north_support_beams(data):
    rounds, flats, height, width = parse_data(data)
    tilt_north(rounds, flats, height, width)
    return sum_rockloads(rounds, height)

In [140]:
total_load_on_north_support_beams(testdata)

136.0

In [141]:
total_load_on_north_support_beams(puzzle)

108840.0

### Part 2
Empirical, not very pretty


In [203]:
def rotate_east(pts, height, width):
    newpts = set()
    for p in pts:
        c, r = int(p.real), int(-p.imag)
        r1 = c
        c1 = height - 1 - r
        newpts.add(c1 - r1*1j)
    return newpts, width, height


def spincycle(rounds, flats, height, width):
    for _ in range(4):
        tilt_north(rounds, flats, height, width)
        rounds, _, _ = rotate_east(rounds, height, width)
        flats, height, width = rotate_east(flats, height, width)
    return rounds, flats, height, width

In [204]:
%%time
rounds0, flats0, height, width = parse_data(testdata)
rounds, flats, height, width = parse_data(testdata)
L = []
for i in range(1_000):
    L.append(sum_rockloads(rounds, height))
    rounds, flats, height, width = spincycle(rounds, flats, height, width)

CPU times: user 171 ms, sys: 0 ns, total: 171 ms
Wall time: 171 ms


In [201]:
startlen = 3
repeater = [69, 69, 65, 64, 65, 63, 68]
repeatlen = len(repeater)
N = 1_000_000_000
repeater[(N - startlen) % repeatlen]

64

In [194]:
%%time
rounds, flats, height, width = parse_data(puzzle)
L = []
for i in range(1_000):
    L.append(sum_rockloads(rounds, height))
    rounds, flats, height, width = spincycle(rounds, flats, height, width)

CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 11.7 s


In [195]:
L[500]

103557.0

In [196]:
L.index(103557, 501)

559

In [197]:
L.index(103557, 560)

618

In [198]:
L[500:559] == L[559:618]

True

In [199]:
repeater = L[500:559]
repeatlen = len(repeater)
startlen = 500
N = 1_000_000_000
repeater[(N - startlen) % repeatlen]

103445.0